# Cas d'usage analytiques : SQL
Ce notebook exécute des requêtes SQL sur les tables Iceberg (via Spark).

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame

# Configuration Spark avec Iceberg et MinIO
from pyspark.sql import SparkSession
from pyspark import SparkConf
import os

# Credentials MinIO
os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"
os.environ["AWS_REGION"] = "eu-west-1"

spark = SparkSession.builder.appName("IcebergNotebook").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
print("Spark session initialisée avec Iceberg et MinIO.")

spark.sql('CREATE NAMESPACE IF NOT EXISTS lakehouse.gold').show()
print(f"Namespace Iceberg créée : gold")

# 🟡 **1. GOLD — Driver Efficiency**
## 🎯 Objectif
Fournir une vision consolidée de la performance des conducteurs :
- efficacité carburant (MPG)
- distance totale parcourue
- nombre de trajets
- coût carburant total
- classement des conducteurs les plus efficaces

Cette table alimente les dashboards Superset : *Performance Conducteur*.

---

## 🧮 Logique & Calculs

| Champ | Description | Calcul |
|-------|-------------|--------|
| `trips` | Nombre de trajets effectués | `COUNT(trip_id)` |
| `total_miles` | Distance totale parcourue | `SUM(actual_distance_miles)` |
| `avg_mpg` | Consommation moyenne | `AVG(average_mpg)` |
| `fuel_gallons` | Gallons consommés | `SUM(fuel_gallons_used)` |
| `fuel_cost` | Coût carburant | `fuel_gallons * 3.80` |
| `mpg_rank` | Classement efficacité | `RANK() OVER (ORDER BY avg_mpg DESC)` |
| `load_ts` | Timestamp Iceberg | `current_timestamp()` |

Seuls les conducteurs ayant **≥ 50 trajets** sont conservés.

---

## 📊 Visualisation Superset attendue

- **Top 10 Conducteurs les plus efficaces** : table ou KPI list
- **Distribution du MPG moyen** : histogramme
- **Coût carburant par conducteur** : bar chart
- **MPG Rank** : leaderboard

---

## 📁 Source
- `lakehouse.silver.fact_trips`
- `lakehouse.silver.dim_drivers`


In [ ]:
# Performance chauffeur
df = spark.sql("""
    CREATE OR REPLACE TABLE lakehouse.gold.driver_efficiency
    USING iceberg
    AS
    WITH driver_performance AS (
        SELECT 
            t.driver_id,
            d.first_name,
            d.last_name,
            t.trip_id,
            t.actual_distance_miles,
            t.average_mpg,
            t.fuel_gallons_used
        FROM lakehouse.silver.fact_trips t
        LEFT JOIN lakehouse.silver.dim_drivers d 
            ON t.driver_id = d.driver_id
    ),

    driver_metrics AS (
        SELECT
            driver_id,
            first_name,
            last_name,
            COUNT(trip_id) AS trips,
            SUM(actual_distance_miles) AS total_miles,
            AVG(average_mpg) AS avg_mpg,
            SUM(fuel_gallons_used) AS fuel_gallons,
            SUM(fuel_gallons_used) * 3.80 AS fuel_cost
        FROM driver_performance
        GROUP BY driver_id, first_name, last_name
    ),

    ranked AS (
        SELECT
            *,
            RANK() OVER (ORDER BY avg_mpg DESC) AS mpg_rank
        FROM driver_metrics
        WHERE trips >= 50
    )

    SELECT
        driver_id,
        first_name,
        last_name,
        trips,
        avg_mpg,
        total_miles,
        fuel_gallons,
        fuel_cost,
        mpg_rank,
        current_timestamp() AS load_ts
    FROM ranked;""").show()

print("✅ Table 'driver_efficiency' créée avec succès.")

spark.sql("REFRESH TABLE lakehouse.gold.driver_efficiency")
print("✅ Table 'driver_efficiency' rafraîchie.")

# Visualisation
df_viz = spark.sql("""
    SELECT
        first_name,
        last_name,
        avg_mpg,
        trips
    FROM lakehouse.gold.driver_efficiency
    ORDER BY avg_mpg DESC
    LIMIT 20
""").show()

# 🚛 **2. GOLD — Truck Utilization & Efficiency**
## 🎯 Objectif
Mesurer la performance opérationnelle des camions :
- utilisation
- consommation
- coût carburant
- idle vs operating time

---

## 🧮 Logique & Calculs

| Champ | Description | Calcul |
|-------|-------------|--------|
| `trips` | Nombre de trajets | `COUNT(trip_id)` |
| `total_miles` | Miles parcourus | `SUM(actual_distance_miles)` |
| `avg_mpg` | MPG moyen | `AVG(average_mpg)` |
| `fuel_gallons` | Gallons consommés | `SUM(fuel_gallons_used)` |
| `fuel_cost` | Coût carburant | `fuel_gallons * 3.80` |
| `utilization_rate` | Taux d’utilisation | issu de `silver.agg_truck_utilization_metrics` |
| `idle_hours` | Temps en idle | idem |
| `operating_hours` | Temps d’opération | idem |

---

## 📊 Dashboards Superset

- **Utilisation des camions** : gauge chart
- **Coût carburant** : bar chart
- **Idle vs Operating Hours** : donut chart
- **Classement des camions par MPG** : leaderboard

---

## 📁 Sources
- `lakehouse.silver.fact_trips`
- `lakehouse.silver.dim_trucks`
- `lakehouse.silver.agg_truck_utilization_metrics`


In [ ]:
df = spark.sql("""
CREATE OR REPLACE TABLE lakehouse.gold.load_profitability
USING iceberg AS
WITH base AS (
    SELECT
        l.load_id,
        l.customer_id,
        c.customer_name,
        l.booking_date,
        l.revenue,
        t.actual_distance_miles,
        t.fuel_gallons_used,
        (t.fuel_gallons_used * 3.80) AS fuel_cost
    FROM lakehouse.silver.fact_loads l
    LEFT JOIN lakehouse.silver.fact_trips t ON l.load_id = t.load_id
    LEFT JOIN lakehouse.silver.dim_customers c ON l.customer_id = c.customer_id
)
SELECT
    *,
    (revenue - fuel_cost) AS margin,
    current_timestamp() AS load_ts
FROM base;
""")

df.show()

print("✅ Table 'load_profitability' créée avec succès.")

spark.sql("REFRESH TABLE lakehouse.gold.load_profitability")

print("✅ Table 'load_profitability' rafraîchie.")


# 💸 **3. GOLD — Revenue per Load / Profitability**
## 🎯 Objectif
Quantifier la rentabilité par livraison (load) :
- revenu
- coût carburant
- marge
- détails client

---

## 🧮 Calculs

| Champ | Description | Calcul |
|-------|-------------|--------|
| `revenue` | Revenu brut | depuis silver.fact_loads |
| `actual_distance_miles` | Distance réelle | silver.fact_trips |
| `fuel_cost` | Coût carburant | `fuel_gallons_used * 3.80` |
| `margin` | Marge | `revenue - fuel_cost` |

---

## 📊 Dashboards Superset

- **Marge par livraison** : bar chart
- **Top clients par profitabilité** : ranked table
- **Évolution du revenu** : line chart
- **Cartographie des marges par route** : map

---

## 📁 Sources
- `lakehouse.silver.fact_loads`
- `lakehouse.silver.fact_trips`
- `lakehouse.silver.dim_customers`


In [ ]:
df = spark.sql("""
CREATE OR REPLACE TABLE lakehouse.gold.load_profitability
USING iceberg AS
WITH base AS (
    SELECT
        l.load_id,
        l.customer_id,
        c.customer_name,
        l.load_date,
        l.revenue,
        t.actual_distance_miles,
        t.fuel_gallons_used,
        (t.fuel_gallons_used * 3.80) AS fuel_cost
    FROM lakehouse.silver.fact_loads l
    LEFT JOIN lakehouse.silver.fact_trips t ON l.load_id = t.load_id
    LEFT JOIN lakehouse.silver.dim_customers c ON l.customer_id = c.customer_id
)
SELECT
    *,
    (revenue - fuel_cost) AS margin,
    current_timestamp() AS load_ts
FROM base;
""")

df.show()

print("✅ Table 'load_profitability' créée avec succès.")

spark.sql("REFRESH TABLE lakehouse.gold.load_profitability")

print("✅ Table 'load_profitability' rafraîchie.")


# 🛠 **4. GOLD — Maintenance Cost Summary**
## 🎯 Objectif
Analyser le coût et l’impact opérationnel des maintenances camions.

---

## 🧮 Calculs

| Champ | Description | Calcul |
|-------|-------------|--------|
| `maintenance_events` | Nombre d'interventions | `COUNT(maintenance_id)` |
| `parts_cost` | Coût des pièces | `SUM(parts_cost)` |
| `labor_cost` | Coût de la main d'œuvre | `SUM(labor_cost)` |
| `total_cost` | Coût total | `SUM(total_cost)` |
| `avg_downtime` | Durée moyenne d'immobilisation | `AVG(downtime)` |

---

## 📊 Dashboards Superset

- **Top 10 camions les plus coûteux** : bar chart
- **Répartition pièces vs main d’œuvre** : pie chart
- **Downtime moyen** : KPI card
- **Evolution des coûts de maintenance** : line chart

---

## 📁 Sources
- `lakehouse.silver.fact_maintenance_records`
- `lakehouse.silver.dim_trucks`


In [ ]:
df = spark.sql("""
CREATE OR REPLACE TABLE lakehouse.gold.maintenance_costs
USING iceberg AS
SELECT
    m.truck_id,
    t.vin,
    COUNT(m.maintenance_id) AS maintenance_events,
    SUM(m.parts_cost) AS parts_cost,
    SUM(m.labor_cost) AS labor_cost,
    SUM(m.total_cost) AS total_cost,
    AVG(m.downtime_hours) AS avg_downtime,
    current_timestamp() AS load_ts
FROM lakehouse.silver.fact_maintenance_records m
LEFT JOIN lakehouse.silver.dim_trucks t ON m.truck_id = t.truck_id
GROUP BY m.truck_id, t.vin;
""")
df.show()

print("✅ Table 'maintenance_costs' créée avec succès.")

spark.sql("REFRESH TABLE lakehouse.gold.maintenance_costs")
print("✅ Table 'maintenance_costs' rafraîchie.")


# 🛡 **5. GOLD — Safety / Incident Analytics**
## 🎯 Objectif
Monitorer la sécurité des conducteurs :
- incidents
- gravité
- coût associé

---

## 🧮 Calculs

| Champ | Description | Calcul |
|-------|-------------|--------|
| `incident_count` | Nombre total d’incidents | `COUNT(incident_id)` |
| `high_risk_incidents` | Incidents graves | `CASE WHEN severity = 'HIGH' THEN 1` |
| `avg_damage_cost` | Coût moyen | `AVG(damages_cost)` |
| `total_damage_cost` | Coût total | `SUM(damages_cost)` |

---

## 📊 Dashboards Superset

- **Heatmap incidents par conducteur**
- **Répartition des niveaux de sévérité**
- **Coûts d'incident cumulé**
- **Top conducteurs à risque** (table ranking)

---

## 📁 Sources
- `lakehouse.silver.fact_safety_incidents`
- `lakehouse.silver.dim_drivers`


In [ ]:
df = spark.sql("""
CREATE OR REPLACE TABLE lakehouse.gold.safety_summary
USING iceberg AS
SELECT
    s.driver_id,
    d.first_name,
    d.last_name,
    COUNT(s.incident_id) AS incident_count,
    SUM(CASE WHEN description like '%Severe%' THEN 1 ELSE 0 END) AS high_risk_incidents,
    AVG(s.claim_amount) AS avg_damage_cost,
    SUM(s.claim_amount) AS total_damage_cost,
    current_timestamp() AS load_ts
FROM lakehouse.silver.fact_safety_incidents s
LEFT JOIN lakehouse.silver.dim_drivers d ON d.driver_id = s.driver_id
GROUP BY s.driver_id, d.first_name, d.last_name;
""")
    
df.show()

print("✅ Table 'safety_summary' créée avec succès.")

spark.sql("REFRESH TABLE lakehouse.gold.safety_summary")
print("✅ Table 'safety_summary' rafraîchie.")

# 🏭 **6. GOLD — Facility Geo KPIs**
## 🎯 Objectif
Analyser les performances logistiques par installation :
- volume d’événements
- taux de livraison
- géolocalisation

---

## 🧮 Calculs

| Champ | Description | Calcul |
|-------|-------------|--------|
| `events` | Nombre d’événements | `COUNT(event_id)` |
| `delivered` | Livrés | `CASE status = 'DELIVERED'` |
| `delayed` | Retards | `CASE status = 'DELAYED'` |

---

## 📊 Dashboards Superset

- **Carte géographique des installations**
- **Taux de retard par facility**
- **Heatmap sur densité d’événements**
- **Top facilities par volume**

---

## 📁 Sources
- `lakehouse.silver.dim_facilities`
- `lakehouse.silver.fact_delivery_events`


In [ ]:
df = spark.sql("""
          CREATE OR REPLACE TABLE lakehouse.gold.facility_geostats
USING iceberg AS
SELECT
    f.facility_id,
    f.facility_name,
    f.latitude,
    f.longitude,
    COUNT(e.event_id) AS events,
    SUM(CASE WHEN e.event_type = 'DELIVERED' THEN 1 ELSE 0 END) AS delivered,
    SUM(CASE WHEN e.event_type = 'DELAYED' THEN 1 ELSE 0 END) AS delayed,
    current_timestamp() AS load_ts
FROM lakehouse.silver.dim_facilities f
LEFT JOIN lakehouse.silver.fact_delivery_events e 
    ON f.facility_id = e.facility_id
GROUP BY f.facility_id, f.facility_name, f.latitude, f.longitude;
""")
df.show()

print("✅ Table 'maintenance_costs' créée avec succès.")

spark.sql("REFRESH TABLE lakehouse.gold.facility_geostats")
print("✅ Table 'facility_geostats' rafraîchie.")

# 🚚 **7. GOLD — Route Performance**

## 🎯 Objectif
Évaluer la performance logistique par route :
- utilisation
- efficacité carburant
- distances

---

## 🧮 Calculs

| Champ | Description | Calcul |
|-------|-------------|--------|
| `trips` | Trajets | `COUNT(trip_id)` |
| `avg_miles` | Miles moyens | `AVG(actual_distance_miles)` |
| `avg_mpg` | MPG moyen | `AVG(average_mpg)` |
| `fuel_used` | Fuel total utilisé | `SUM(fuel_gallons_used)` |
| `total_miles` | Miles totaux | `SUM(actual_distance_miles)` |

---

## 📊 Dashboards Superset

- **Top routes utilisées** : bar chart
- **MPG moyen par route** : line chart
- **Distance parcourue par route** : table / KPI
- **Carte Origine → Destination** : arrows map

---

## 📁 Sources
- `lakehouse.silver.fact_trips`
- `lakehouse.silver.dim_routes`


In [ ]:
df = spark.sql("""
CREATE OR REPLACE TABLE lakehouse.gold.route_performance
USING iceberg AS
WITH trip_loads AS (
    SELECT 
        t.trip_id,
        l.load_id,
        l.route_id,
        t.actual_distance_miles,
        t.average_mpg,
        t.fuel_gallons_used
    FROM lakehouse.silver.fact_trips t
    LEFT JOIN lakehouse.silver.fact_loads l
        ON t.load_id = l.load_id
)
SELECT
    r.route_id,
    r.origin_city,
    r.destination_city,
    COUNT(t.trip_id) AS trips,
    AVG(t.actual_distance_miles) AS avg_miles,
    AVG(t.average_mpg) AS avg_mpg,
    SUM(t.fuel_gallons_used) AS fuel_used,
    SUM(t.actual_distance_miles) AS total_miles,
    current_timestamp() AS load_ts
FROM trip_loads t
LEFT JOIN lakehouse.silver.dim_routes r
    ON r.route_id = t.route_id
GROUP BY r.route_id, r.origin_city, r.destination_city
""")

df.show()
print("✅ Table 'route_performance' créée avec succès.")

# Rafraîchir d’autres tables si nécessaire
spark.sql("REFRESH TABLE lakehouse.gold.route_performance")
print("✅ Table 'facility_geostats' route_performance.")


# 📦 **Récap — GOLD tables générées** 

| Gold Table | Description | 
| -------------------- | ------------------------------------------------ | 
| driver_efficiency | Ranking et performance carburant des conducteurs | 
| truck_efficiency | Utilisation et performances des camions | 
| load_profitability | Marge, coûts carburant et revenu par livraison | 
| maintenance_costs | Coûts & downtime maintenance | 
| safety_summary | Incidents, risques, coûts | 
| facility_geostats | Analyse géospatiale des installations | 
| route_performance | Performances et statistiques des routes |